In [1]:
if True: # enable folding code
    if False:
        from julia.api import Julia
        jl = Julia(compiled_modules=False)

    import julia; julia.install(quiet=True)
    from julia import Main

    import numpy as np
    import sympy as sp
    from IPython.display import display, Math

    np.set_printoptions(precision=3, suppress=True)

    try:
        from scipy.linalg import cholesky, eigh, qz, ordqz
        HAVE_SCIPY = True
    except Exception as e:
        HAVE_SCIPY = False
        print("SciPy not available; QZ and ordqz demos will be skipped:", e)

In [2]:
%load_ext julia.magic

Initializing Julia interpreter. This may take some time...


In [3]:
%%julia
using Pkg
gla_dir = "../GenLinAlgProblems"
Pkg.activate(gla_dir)
using GenLinAlgProblems, LinearAlgebra, BlockArrays, RowEchelon, LaTeXStrings, Latexify, SymPy, Random

using PyCall
itikz = pyimport("itikz")
nM    = pyimport("itikz.nicematrix");

function is_upper_hessenberg(H; tol = 1e-10)
    n = size(H, 1)
    for i in 3:n
        for j in 1:i-2
            if abs(H[i,j]) > tol
                return false
            end
        end
    end
    return true
end

function is_upper_triangular(T; tol = 1e-10)
    n = size(T, 1)
    for i in 2:n
        for j in 1:i-1
            if abs(T[i,j]) > tol
                return false
            end
        end
    end
    return true
end
;

  Activating project at `C:\Users\jeff\NOTEBOOKS\elementary-linear-algebra\GenLinAlgProblems`
Precompiling packages...
   2798.6 ms  ✓ Conda
   8012.6 ms  ✓ PlotlyBase
   5433.3 ms  ✓ SymPy
   5584.3 ms  ✓ PyPlot
  20131.5 ms  ✓ GenLinAlgProblems
  5 dependencies successfully precompiled in 64 seconds. 119 already precompiled.
  1 dependency precompiled but a different version is currently loaded. Restart julia to access the new version. Otherwise, loading dependents of this package may trigger further precompilation to work with the unexpected version.

<div style="height:2cm;">
<div style="float:center;width:100%;text-align:center;"><strong style="height:100px;color:darkred;font-size:40px;">Generalized Eigenvalue Computation</strong>
</div></div>

# 1. Introduction

This notebook addresses numerical methods for solving the generalized eigenproblem (GEP):  
$\qquad \displaystyle{ A \mathbf{v} = \lambda B \mathbf{v} }$

The focus is on algorithmic strategies for computing eigenvalues and eigenvectors,  
particularly in structured cases where $A$ and $B$ are symmetric, and $B$ is positive definite.  
Structured solvers are contrasted with general-purpose routines suitable for arbitrary pencils.

Topics include:

- Solvers for symmetric-definite pencils
- Treatment of general matrix pencils
- Sensitivity to conditioning and numerical roundoff
- Interpretation of computed eigenpairs

The theoretical foundation, including variational formulations and physical interpretations,  
is presented in the preceding notebook: [**GEP_intro.ipynb**](GEP_intro.ipynb).

____
Below, we introduce relevant algorithms to solve the GEP problem using simple reference implementations.

# 2. Symmetric-Definite Case

Consider the generalized eigenproblem  
$\qquad \displaystyle{ A x = \lambda B x}$  
where both matrices $A$ and $B$ are real symmetric, and $B$ is **positive definite**,  
i.e., $A=A^T$, $B=B^T \succ 0$

**Remark:** when $B$ is invertible, we may convert the GEP to a standard eigenproblem:  
$\qquad A x = \lambda B x \;\Leftrightarrow\; B^{-1} A x = \lambda x$.  
$\qquad$ Numerically however, forming $B^{-1} A$ explicitly is **dangerous** when $B$ is ill-conditioned;
it also destroys structure.  

$\qquad$ We will therefore formulate an algorithm that avoids inverting $B$ using the Cholesky Factorization.

____

Since $B$ is symmetric positive definite, it admits a **Cholesky decomposition**:  
$\qquad \displaystyle{ B = L^T L }$  
where $L$ is upper triangular with positive diagonal entries.

Substituting into the generalized eigenproblem:  
$\qquad \displaystyle{ A x = \lambda\ L^T L x }$

Let $\;\; \displaystyle{L x = z \Leftrightarrow x = L^{-1} z,\;\; }$ then  
$\qquad \displaystyle{
\begin{aligned}
A x = \lambda B x 
\quad &\Leftrightarrow\quad A L^{-1} z = \lambda L^T L L^{-1} z \\
     &\Leftrightarrow\quad A L^{-1} z = \lambda L^T z \\
     &\Leftrightarrow\quad (L^T)^{-1} A L^{-1} z = \lambda z
\end{aligned}
}$

Define $\;\; \displaystyle{ \tilde{A} = (L^T)^{-1} A L^{-1} }$.  
Then the problem reduces to the standard eigenproblem:  
$\qquad \displaystyle{ \tilde{A} z = \lambda z }$

Once the eigenpairs $(\lambda, z)$ are computed, the original eigenvectors are obtained by  
$\qquad \displaystyle{ x = L^{-1} z }$

**Remark:** **explicit computation of** $\displaystyle{ L^{-1} }$ **or** $\displaystyle{ (L^T)^{-1} }$ **should be avoided in practice**.
- To evaluate $\displaystyle{x = L^{-1} z }$, compute $z$ via forward substitution (i.e., solve $L x = z$)
- To apply the transformation $\displaystyle{ \tilde{A} = (L^T)^{-1} A L^{-1} }$,
  solve two triangular systems rather than forming the inverse

#### **Algorithm Summary**

1) Compute $L$ (Cholesky factorization of $B$).  
2) Form $\tilde A = L^{-1} A L^{-T}$ by triangular solves.  
3) Compute eigenpairs $(\lambda, z)$ of the symmetric matrix $\tilde A$.  
4) Recover $x = L^{-T} z$; optionally $B$-normalize: $x \leftarrow x/\sqrt{x^T B x}$.

In [4]:
def gep_symmetric_definite(A,B):
    """Solve A x = λ B x for symmetric A and SPD B via Cholesky whitening.
    Returns (lam, X) with columns of X being B-normalized eigenvectors.
    """
    if not HAVE_SCIPY:
        raise RuntimeError("SciPy required here for robust Cholesky/eigh.")
    L = cholesky(B, lower=True)
    # Solve for At = L^{-1} A L^{-T}
    At = np.linalg.solve(L, A)
    At = At @ np.linalg.solve(L.T, np.eye(A.shape[0]))  # Note we are avoiding computing an inverse by solving Lᵀ X = I, not inv( Lᵀ )
    lam, Z = eigh(At)
    # Map back and B-normalize
    X = np.linalg.solve(L.T, Z)
    for i in range(X.shape[1]):
        xi = X[:,i]
        nB = np.sqrt(xi.T @ B @ xi)
        if nB > 0:
            X[:,i] = xi / nB
    return lam, X

#### **Example**

In [5]:
def sym_pos_def_demo():
    #np.random.seed(0)
    n = 5
    R = np.random.randn(n,n)
    B = R@R.T + n*np.eye(n)       # SPD
    S = np.random.randn(n,n)
    A = 0.5*(S+S.T)
    lam, X = gep_symmetric_definite(A,B)

    Main.py_show("A = ", np.round(A,2), r",\quad B = ", np.round(B,2));

    print("\nEigenvalues (symmetric-definite route):", lam)
    print("\n    ||X^T B X - I||         = ", np.linalg.norm(X.T @ B @ X - np.eye(n)))
    print("    ||A X - B X diag(lam)|| = ", np.linalg.norm(A@X - B@X@np.diag(lam)))
sym_pos_def_demo()

<IPython.core.display.Latex object>


Eigenvalues (symmetric-definite route): [-0.683 -0.101  0.03   0.112  0.201]

    ||X^T B X - I||         =  8.54768620809237e-16
    ||A X - B X diag(lam)|| =  3.519081438737461e-15


# 3. General Case

To prepare for the general case QZ algorithm, some backgound material is necessary:
* A form of the matrices that is easy to solve: the **Generalized Schur Form**
* Transformations that will be used to reduce a matrix pair $(A,B)$ to this form
* The QZ algorithm in 3 steps: preparation, reduction to an intermediate form, and reduction to the generalized schur form

## 3.1 Matrices in Generalized Schur Form

<div style="background-color:#F2F5A9;color:black;">

**Definition:**  A pair $(S,T)$ is in **(real, quasi-) generalized Schur form**<br>
$\qquad$ if both are block upper triangular with the **same** block pattern<br>
$\qquad$ and each diagonal block is either $1\times1$ or $2\times2$.

The generalized eigenproblem for $(S,T)$ is $\;\; S x=\lambda\,T x$.
</div>

**This immediately yields eigenvalues:**  
Since $(S,T)$ are triangular by blocks, $\det(S-\lambda T)$ factors over the diagonal blocks:
- For a $1\times1$ block $([s_{ii}],[t_{ii}])$, eigenvalue $\,\lambda_i=s_{ii}/t_{ii}$ (finite if $t_{ii}\neq 0$, infinite if $t_{ii}=0$).
- For a $2\times2$ block $(S_{kk},T_{kk})$, the two eigenvalues are the roots of
<br>$\qquad \det(S_{kk}-\lambda T_{kk})=0$.

**This yields eigenvectors.**  
Fix an eigenvalue $\lambda$. Solve the homogeneous system
<br>$\qquad (S-\lambda T)x=0$
by **block back substitution**:
- If the trailing active block is $1\times1$, pick and normalize $x_i$, then solve upward.
- If it is $2\times2$, solve $(S_{kk}-\lambda T_{kk})\,x_{[k:k+1]}=0$, then continue upward.
Left eigenvectors satisfy $w^T S=\lambda\, w^T T$ and are obtained the same way on the transposed pair.

#### Example

Consider
$\;\;
S=\left(\begin{array}{rrrr}
\color{red}4&\color{red}7&5&6\\
\color{red}{-4}&\color{red}{-2}&3&8\\
0&0&\color{red}6&2\\
0&0&0&\color{red}{-3}
\end{array}\right),\quad
T=\begin{pmatrix}
\color{red}2&\color{red}1&4&3\\
\color{red}0&\color{red}2&7&5\\
0&0&\color{red}2&1\\
0&0&0&\color{red}1
\end{pmatrix}\;\;
$ with the 3 blocks shown in red.

**Eigenvalues.**
- Leading $2\times2$ block $\;\;S_{1} = \left(\begin{array}{rr} 4&7 \\ {-4}&{-2}\\ \end{array}\right), \;\;
T_{1} = \begin{pmatrix} 2 & 1 \\ 0 & 2\end{pmatrix}$

$\qquad \det\!\big(S_{1}-\lambda T_{1}\big)=4(\lambda^2-2\lambda+5)=0
\;\Rightarrow\; \lambda=1\pm 2i.$
- $1\times1$ blocks $\;\;S_2=\begin{pmatrix}6\end{pmatrix},\;\; T_2 = \begin{pmatrix}2\end{pmatrix}\;\;$ and
$S_3=\begin{pmatrix}-3\end{pmatrix},\;\; T_3 = \begin{pmatrix}1\end{pmatrix}\;\;$

$\qquad \lambda_3=6/2=3,\quad \lambda_4=-3/1=-3.$

**Right eigenvectors (Schur basis).** For each listed $\lambda$, solve
<br>$\qquad (S-\lambda T) x = 0$
by back substitution as above:

$\qquad
x_{1-2i} =
\left(\begin{array}{c}
-1 + i\\
1\\
0\\
0
\end{array}\right),
\;\;
x_{1+2i} =
\left(\begin{array}{c}
-1 - i\\
1\\
0\\
0
\end{array}\right),
\;\;
x_{3} =
\left(\begin{array}{r}
-16\\
-1\\
4\\
0
\end{array}\right),
\;\;
x_{-3} =
\left(\begin{array}{r}
59\\
-97\\
-20\\
48
\end{array}\right).
$

## 3.2 The A = RQ and the A = QR Factorizations

To transform a square matrix pair to generalized Schur Form, we will use two operations that introduce zeros in rows/columns of a matrix<br>
using orthogonal matrices:

Orthogonal matrices $Q$ from the left or $Z$ from the right can be chosen to annihilate selected entries in a matrix $A$<br>
while preserving numerical stability:
* $Q A = R$ the matrix $Q$ forms linear combinations of the rows of $A$. We can choose $Q$ to introduce 0 in a particular column of the matrix $A$
* $A Z = R$ the matrix $A$ forms linear combinations of the columns of $A$. We can choose $Z$ to introduce 0 in a particular row of the matrix $A$

In the QR_step!() and RQ_step!() operations, this idea is localized: each step constructs a Householder reflection that introduces zeros<br>
in a specific column or row of a given matrix respectively.

These reflections yield compact, stable transformations that maintain orthogonality throughout the reduction process.<br>
While this implementation uses Householder reflections, the same purpose can be achieved using Givens rotations,<br>
which apply successive plane rotations to eliminate entries individually.

### 3.2.1 Householder Reflections

Householder reflections were first introduced in [**HouseholderReflections**](HouseholderReflections)

<div>
<div style="float:left;padding-right:2cm;padding-top:0cm;">
Let $x \in \mathbb{R}^n$ and let $e$ be a unit vector defining the target direction.<br>
Define $y = x - \Vert x \Vert e$, which determines the reflection that maps $x$ onto $e$.<br>
Decompose $x$ along $y$ as $x = x_\parallel + x_\perp$,<br>
where $x_\parallel = \frac{y^T x}{y^T y} y$ is the component of $x$ parallel to $y$.<br>
The reflected vector is<br>
$\qquad
\Vert x \Vert e = x - 2x_\parallel = x - 2\frac{y^T x}{y^T y} y.
$

The corresponding orthogonal and symmetric Householder matrix is<br>
$\qquad
H = I - 2\frac{y\ y^T}{y^T y},
$

which satisfies $H x = \Vert x \Vert e$ and preserves vector norms under reflection.
</div>
<div style="float:left;width:30%;"><img src="./Figs/HouseholderReflection_v1.svg" width=600></div>
</div>

In [6]:
%%julia
@doc """
householder_vector(x::AbstractVector; target::Symbol = :first) -> Vector

Computes a unit length Householder vector `w` such that the reflection
    `Q = I - 2wwᵀ` maps `x` to a vector aligned with the `target` coordinate vector :first or :last

Arguments:
- `x`: A vector to be reflected.
- `target`: `:first` (default) aligns with the first coordinate; `:last` aligns with the last.

Returns:
- A unit Householder vector `w` such that `Q x` has all zeros except possibly the first or last entry.
"""
function householder_vector(x::AbstractVector; target::Symbol = :first)
    x = vec(x)
    if all(iszero, x)
        return zeros(eltype(x), length(x))
    end

    σ        = norm(x)
    e        = zeros(eltype(x), length(x))
    index    = target == :first ? 1 : length(x)
    e[index] = sign(x[index]) == 0 ? σ : sign(x[index]) * σ  # the sign determines which of the two possible bisectors to use

    w     = x - e
    wnorm = norm(w)

    if wnorm ≈ 0
        return zeros(eltype(x), length(x))
    end

    result = w / wnorm
    return result
end
;

In [7]:
%%julia
function test_householder_vector(x; target::Symbol=:first)
   w = householder_vector(float.(x), target=target)
   Q = 1.0I - 2w*w'
   py_show( "first = ", target==:first ? "true" : "false", L",\qquad x=", Int.(x), L",\qquad w = ", w, L",\quad Q = ", Q, L"\qquad Q x = ", Q*x, number_formatter=x->round(x,digits=2))
   @show ((Q'Q ≈ I)); flush(stdout)
end
println( "Reflecting vector [1,1,1] onto e_1 and onto e_3")
test_householder_vector( [ 1,1,1 ] )
test_householder_vector( [ 1,1,1 ]; target=:last  )

Reflecting vector [1,1,1] onto e_1 and onto e_3

<IPython.core.display.Latex object>


Q' * Q ≈ I = true


<IPython.core.display.Latex object>

Q' * Q ≈ I = true


____
Below, we will use **Householder Reflections to introduce zeros in a row and/or a column** of a matrix.

### 3.2.2 $A = QR$ one Column at a Time, $A = RZ$ one Row at a Time

We will need to zero out entries in a matrix one column and/or one row at a time.<br>

Zeroing out entries in a column can be done by using a Householder reflector matrix $H$ from the left. The transformation<br>
$\qquad \tilde{A} = H A$
uses $H$ to form linear combinations of the rows of $A$, creating zeros in the column targeted by $H$.

Below a naive reference implementation using **Householder reflections** to zero out entries<br>
$\qquad$ **one column at a time**.

**Remark:** A similar routine could be implemented using **Givens Rotations.**

In [8]:
%%julia
"""
    QR_step!(A::Matrix{Float64}, i::Int, j::Int) -> Matrix

Applies a Householder reflection **from the left** to zero out entries
**below** the pivot `A[i,j]` in column `j`. That is, it modifies `A`
in-place as `A ← Q * A`, where `Q` is orthogonal.

Returns the orthogonal matrix `Q` such that the updated `A = Q * A`
has zeros in entries A[i+1:end, j].

Arguments:
- `A`: Matrix to modify in-place.
- `i`: Row index of the pivot.
- `j`: Column index of the pivot.

Returns:
- The orthogonal matrix `Q` such that `A ← Q * A`.
"""
function QR_step!(A::AbstractMatrix{Float64}, i::Int, j::Int)
    m = size(A, 1)

    # Extract column segment below and including the pivot
    x     = copy(@view A[i:end, j])
    v_sub = householder_vector(x; target = :first)  # Align with first coordinate

    v = zeros(eltype(A), m)
    v[i:end] .= v_sub

    Q = I - 2 * (v * v')  # Householder matrix
    A .= Q * A            # Apply from the left

    return Q
end;

Similarly, zeroing out a entries in a row can be done by using a Householder reflector matrix $H$ from the right. The transformation<br>
$\qquad \tilde{A} = A H$
uses $H$ to form linear combinations of the columns of $A$, creating zeros in the row targeted by $H$.

Below a naive reference implementation using **Householder reflections** to zero out entries<br>
$\qquad$ **one row at a time**

In [9]:
%%julia

function RQ_step!(
    A::AbstractMatrix{Float64},
    i::Int,              # pivot row index
    j::Int;              # pivot column index
    len::Int = j         # number of active entries (default: full 1:j)
)
    n = size(A, 2)
    if len <= 0
        return Matrix{Float64}(I, n, n)
    end

    start_col = max(1, j - len + 1)
    x = @view A[i, start_col:j]          # read directly from A

    v_sub = householder_vector(x; target = :last)

    v = zeros(eltype(A), n)
    v[start_col:j] .= v_sub

    Qr = I - 2 * (v * v')
    A .= A * Qr
    return Qr
end
;

These two operations can be combined to obtain factorizations of $A$
* $A = Q R\_l$ consists of $Q$, a product of orthogonal matrices zeroing out one column at a time, from left to right<br>
and an upper triangular matrix $R\_l$
* $A = R\_r Z$ consists of $Z$, a product of orthogonal matrices zeroing out one row at a time, from bottom to top<br>
and an upper triangular matrix $R\_r$

In [10]:
%%julia
function test_qr_and_rq(n::Int = 4)
    A = randn(n, n)
    Rl = copy(A)
    Rr = copy(A)
    py_show("Convert A to upper triangular: ", L"A = Q R_l\;\;", "and", L"\;\; A = R_r Z" )

    display(py_show("original matrix A = ", A, number_formatter = x -> round(x, digits=2)))

    Q = Matrix{Float64}(I, n, n)
    Z = Matrix{Float64}(I, n, n)
    for j in 1:n
        Qj     = QR_step!(Rl, j, j)
        Zj     = RQ_step!(Rr, n-j+1, n-j+1)
        Q      = Qj * Q  # Accumulate from left
        Z      = Z * Zj  # Accumulates from right
        py_show(L"R_l = ",      (m=Rl, per_element_style = (x,ii,jj,fx)->tril_formatter(x,ii,jj,fx; k=-1, c1=1,c2=j)),
                L"\qquad R_r =",(m=Rr, per_element_style = (x,ii,jj,fx)->tril_formatter(x,ii,jj,fx; k=-1, r1=n-j+1,r2=n)),
                        number_formatter = x -> round(x, digits=2))
    end

    @show (A ≈ Q' * Rl)           # A = Qᵗ R → Q R ≈ A
    @show (A ≈ Rr * Z')
    flush(stdout)
    @show (Q'Q ≈ I)
    @show (Z'Z ≈ I)
    flush(stdout)
end
test_qr_and_rq(4)

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

A ≈ Q' * Rl = true
A ≈ Rr * Z' = true
Q' * Q ≈ I = true
Z' * Z ≈ I = true


* On the left, we form successive Householder reflection matrices using the $x$ vector from the diagonal down,<br>
introducing zeros in each successive column (from left to right).<br>
The reflector is chosen to maintain a diagonal value (QR_step with target=:first)
* On the right, we form successive Householder reflection matrices using the $x$ vector from the left to the current diagonal entry,<br>
introducing zeros in each successive row (from the bottom up).<br>
The reflector is chosen to maintain a diagonal value (RQ_step with target=:last)

## 3.3 Reducing a Pencil to Hessenberg Triangular Form

### 3.3.1 Hessenberg–Triangular Reduction

The **Hessenberg–triangular (HT) reduction** transforms a matrix pair $(A,B)$ into an equivalent pair $(H,T)$, where $H$ is upper Hessenberg and $T$ is upper triangular:

$\qquad
Q^{T} A Z = H, \qquad Q^{T} B Z = T.
$

---

#### **Step 1.** Begin by reducing $B$ to upper triangular form using an orthogonal matrix from the left:

> Triangularize $B$ using the QR decomposition:
> 
> $\qquad$   Apply a sequence of $Q_k$ (“QR steps”) to $B$ until it becomes upper triangular.<br>
$\qquad$   The orthogonal matrices are applied to both matrices $A$ and $B$.
> 
>   $\qquad\qquad
   B \leftarrow Q_k B, \qquad  A \leftarrow Q_k A.
   $

#### **Step 2.** Form Hessenberg–triangular pair (optional step):

>   For each column $k = 1,\dots,n-2$:
>   - Use a *left* orthogonal matrices $Q_k$ to zero entries below $A_{k+1,k}$;
     (i.e., drive $A$ to upper Hessenberg form one column at a time)
     apply it to both $A$ and $B$.<br>
   $\qquad
   A \leftarrow Q_k A, \qquad  B \leftarrow Q_k B.
   $
>
>   - The left update reintroduces non-zero entries in $B$ (a *bulge* just below the diagonal of $B$.)  
     Eliminate this bulge by applying *right* orthogonal matrices $Z_i$ **from bottom to top**,  
     with the reflection length decreasing upward:
>
>     $\qquad
     i = n, n-1, \dots, k+2, \qquad
     Z_i \text{ acts on row } i \text{ with length } (i-k).
     $
>
>     Each $Z_i$ removes one subdiagonal element of $B$ while updating $A$:
>
>     $\qquad
     B \leftarrow B Z_i, \qquad  A \leftarrow A Z_i.
     $
>
>   After $n-2$ iterations,  
>   $A$ is upper Hessenberg, $B$ is upper triangular, and  
>   the accumulated matrices $Q=\prod Q_k$, $Z=\prod Z_i$ satisfy  
>
>   $\qquad
   H = Q^{T} A Z, \qquad T = Q^{T} B Z.
   $

---
This two-sided process alternates left “QR” reductions and right “RQ” *bulge chases*,<br>
progressively constructing the upper‑Hessenberg/upper‑triangular pair<br>
that serves as the starting point of the **QZ (Generalized Schur) algorithm**.

In [15]:
%%julia
"""
Educational Hessenberg–Triangular reduction for a matrix pair (A,B). (Steps 1 and 2)

Uses:
  • QR_step!(A, i, j) – left Householder; zeros entries below A[i,j].
  • RQ_step!(A, i, j) – right Householder; zeros entries left of A[i,j].

Both functions modify the given matrix in place and return the full N×N orthogonal Q or Z.

Performs:
    Qᵗ * A * Z = H
    Qᵗ * B * Z = T
where H is upper Hessenberg and T is upper triangular.
"""
function hessenberg_triangular_reduction!(A::Matrix{Float64}, B::Matrix{Float64}; debug::Bool = true)
    n = size(A, 1)
    Q_total = Matrix{Float64}(I, n, n)
    Z_total = Matrix{Float64}(I, n, n)

    # ---------------------------------------------------------
    # Step 1 : Triangularize B (QR-type left transformations)
    # ---------------------------------------------------------
    if debug println("=========== Step 1 : S=A, T=B; Triangularize B from the left (QR_steps)");flush(stdout) end
    for j in 1:n-1
        Q = QR_step!(B, j, j)      # internally: B ← Q B
        A .= Q * A                 # apply Q to A once
        Q_total .= Q * Q_total
    end
    if debug py_show("S = ",  (a=A,), L"\quad T = ",
             (b=B, per_element_style = (x,i,j,fx)->tril_formatter(x,i,j,fx; k=-1, c2=n) ),
             number_formatter=x->round(x,digits=2)
    ) end

    # ---------------------------------------------------------
    # Step 2 : Hessenberg–Triangular Reduction
    # ---------------------------------------------------------
    if debug println("\n=========== Step 2 : Hessenberg–Triangular Reduction ===");flush(stdout) end
    for k in 1:n-2
        # ---- Left reflector Qₖ: zero below A[k+1,k]
        if debug println("Make column $k of A upper Hessenberg:");flush(stdout) end
        Qk = QR_step!(A, k+1, k)   # internally: A ← Qₖ A
        B .= Qk * B
        Q_total .= Qk * Q_total

        if debug py_show( "S = ", (a=A, per_element_style = (x,i,j,fx)->tril_formatter(x,i,j,fx;k=-2, c2=k) ), L"\quad",
                 "T = ", (b=B, per_element_style = (x,i,j,fx)->tril_formatter(x,i,j,fx; k=-1) ),
                     number_formatter=x->round(x,digits=2)
        ) end

        # ---- Right reflectors Z : clear left‑of‑diagonal entries in B
        # Start one row below the current pivot (e.g. (3,2) when k = 1)
        for i in n:-1:(k+2)
            len_i = i - k         # shrinking active length as we go up
            Z = RQ_step!(B, i, i; len=len_i)
            A .= A * Z
            Z_total .= Z_total * Z
            if debug
                println("        Remove bulge in T in row  $i (len=$len_i):");flush(stdout)
                py_show( L"\qquad\qquad S = ", (a=A, per_element_style = (x,i,j,fx)->tril_formatter(x,i,j,fx;k=-2, c2=k) ), L"\quad",
                         "T = ", (b=B, per_element_style = (x,i,j,fx)->tril_formatter(x,i,j,fx; k=-1) ),
                         number_formatter=x->round(x,digits=2)
                )
            end
        end
    end

    return Q_total', Z_total
end
;

In [12]:
%%julia
function test_hessenberg_triangular_qz(n=4)
    S = randn(n, n)
    T = randn(n, n)
    A = copy(S)
    B = copy(T)

    Q, Z = hessenberg_triangular_reduction!(S, T)

    @show (A ≈ Q*S*Z')
    @show (B ≈ Q*T*Z')
    flush(stdout)

    @show (Q'Q ≈ I)
    @show (Z'Z ≈ I)
    flush(stdout)

    @show is_upper_hessenberg(S)
    @show is_upper_triangular(T)
    flush(stdout)
end

test_hessenberg_triangular_qz(4);

=========== Step 1 : S=A, T=B; Triangularize B from the left (QR_steps)


<IPython.core.display.Latex object>


=========== Step 2 : Hessenberg–Triangular Reduction ===
Make column 1 of A upper Hessenberg:


<IPython.core.display.Latex object>

        Remove bulge in T in row  4 (len=3):


...


Make column 2 of A upper Hessenberg:


<IPython.core.display.Latex object>

        Remove bulge in T in row  4 (len=2):


<IPython.core.display.Latex object>

A ≈ Q * S * Z' = true
B ≈ Q * T * Z' = true
Q' * Q ≈ I = true
Z' * Z ≈ I = true
is_upper_hessenberg(S) = true
is_upper_triangular(T) = true


____
**Observe:**
* Introducing zeros in a column of $A$ (step 2) to drive it to upper Hessenberg form<br>
reintroduces non-zero entries in $B$
* these non-zero entries are removed from the bottom up, using orthogonal matrices that do not affect the zero columns to the right of the bulge.<br>
This keeps the zeros in $A$ intact.

### 3.3.2 QZ Iteration

The QZ algorithm reduces a matrix pencil $(A,B)$ to its generalized (quasi-)Schur form $(S,T)$,<br>
$\qquad$ in which the generalized eigenvalues are read directly from the diagonal blocks.<br>
$\qquad$ It preserves the spectrum and records transformations so that eigenvectors can be recovered from $(S,T)$.

The **QZ algorithm** generalizes the [**Schur decomposition**](Schur_Decomposition.ipynb) by repeatedly applying a similarity transform<br>
$\qquad$ obtained from the QR decomposition of $A = Q R$, i.e., computing $Q^T A Q = R Q,\;\; Q^T B Q$.<br>
$\qquad$ Each iteration step reintroduces a bulge in the $B$ matrix that is removed as before.

____
It therefore obtains two orthogonal (or unitary) matrices $Q$ and $Z$ such that<br>
$\qquad S = Q^H A Z$ and $T = Q^H B Z = T$, where $(S,T)$

$\qquad$ The spectrum is preserved, and the transformations $Q,Z$ allow recovery of eigenvectors from $(S,T)$:

$\qquad 
\begin{align}
A x = \lambda B x \quad & \Leftrightarrow \quad  Q A Z Z^T\ x = \lambda\ Q B Z Z^T\ x & \\
                        & \Leftrightarrow \quad  Q A Z \ \tilde{x} = \lambda\ Q B Z \ \tilde{x} \qquad\qquad & \text{ where } \tilde{x} = Z^T x
\end{align}$

$\qquad$ Hence $(\lambda, x)$ is a generalized eigenpair of $ A x = \lambda B x$<br>
$\qquad$ if and only if $( \lambda, Z^T x )$ is a generalized eigenpair of $ Q A Z \ \tilde{x} = \lambda \ Q B Z\ \tilde{x}$

In [13]:
%%julia

"""
Perform basic unshifted QZ eigenvalue iterations on a matrix pair (A,B) (optionally in Hessenberg–triangular form).

Each step executes:
    A = Q*R
    A ← R*Q  (so A ← Qᵀ*A*Q)
    B ← Qᵀ*B*Q
then applies an optional bottom‑up RQ sweep to keep B upper‑triangular.

Args:
    A, B : square Float64 matrices (A Hessenberg‑like, B upper‑triangular)
    nsteps : number of outer iterations
    debug : print intermediate results

Returns:
    A, B, Qacc, Zacc : updated matrices and accumulated orthogonal transforms
"""
function qz_eigen_iteration!(A::Matrix{Float64}, B::Matrix{Float64},
                             nsteps::Int=1; debug::Bool=true)
    n = size(A,1)
    Q_total = Matrix{Float64}(I,n,n)
    Z_total = Matrix{Float64}(I,n,n)

    for step in 1:nsteps
        if debug println("=========== QZ Eigen Iteration step $step ============="); flush(stdout) end

        # ---------------------------------------------------------
        # Step 1 : Left QR similarity transform
        if debug println("\n-- Left QR similarity update (A = Q*R; A ← R*Q, B ← Qᵀ B Q)") end

        F = qr(A)
        Q = Matrix(F.Q)
        R = Matrix(F.R)

        A .= R * Q
        B .= Q' * B * Q
        Q_total .= Q' * Q_total     # new Q acts on the left  → prepend
        Z_total .= Z_total * Q     # same Q acts on the right → append

        if debug
            py_show("S = ",
                (a=A, per_element_style=(x,i,j,fx)->tril_formatter(x,i,j,fx;k=-2)),
                L"\quad","T = ",
                (b=B, per_element_style=(x,i,j,fx)->tril_formatter(x,i,j,fx;k=-1)),
                number_formatter=x->round(x,digits=2)
            ); flush(stdout)
        end

        # ---------------------------------------------------------
        # Step 2 : Right RQ steps to restore B to upper‑triangular form
        if debug println("\n-- Right RQ sweep to restore T (upper triangular)"); flush(stdout) end

        for i in n:-1:2
            Z = RQ_step!(B, i, i; len=i)
            A .= A * Z
            Z_total .= Z_total * Z
            if debug
                println("   Remove bulge from T in row $i (len=$i):"); flush(stdout)
                py_show(
                    L"\qquad S=", (a=A, per_element_style=(x,p,q,fx)->tril_formatter(x,p,q,fx;k=-2)),
                    L"\quad T=", (b=B, per_element_style=(x,p,q,fx)->tril_formatter(x,p,q,fx;k=-1)),
                    number_formatter=x->round(x,digits=2)
                ); flush(stdout)
            end
        end
        if step==100
            py_show(L"S = ",
                (a=A, per_element_style=(x,i,j,fx)->tril_formatter(x,i,j,fx;k=-2)),
                L"\quad T = ",
                (b=B, per_element_style=(x,i,j,fx)->tril_formatter(x,i,j,fx;k=-1)),
                number_formatter=x->round(x,digits=2)
            ); flush(stdout)
        end
    end

    return A, B, Q_total', Z_total
end
;

In [16]:
%%julia
function test_qz_iteration()
    #Random.seed!(1234)

    # Example matrices --- B upper-triangular, A random/Hessenberg-like
    n = 4
    A = [1.0 2 3 4;
         5 6 7 8;
         0.1 9 10 11;
         1   0.1 12 13]

    B = [4.0 0.5 0.1 0;
         0.0 3.0 2.1 0.3;
         2.0 0.0 2.0 0.2;
         1.0 -1.0 0.0 1.0]

    py_show("Initial A = ",A, L"\quad B = ",B)
    λ, X = eigen(A, B)
    py_show( "Generalized eigenvalues: ", round.(λ,digits=2)' )
    println()

    S = copy(A)
    T = copy(B)
    S, T, Qc, Zc = qz_eigen_iteration!(S, T, 1)

    println( "\n.... iterate....\n\n")
    S = copy(A)
    T = copy(B)
    S, T, Q, Z = qz_eigen_iteration!(S, T, 100; debug=false)

    # --- Orthogonality check for accumulated Q and Z
    flush(stdout)
    @show (A ≈ Q*S*Z')
    @show (B ≈ Q*T*Z')
    flush(stdout)
    @show (Q' * Q ≈ I)
    @show (Z' * Z ≈ I);
end
test_qz_iteration();

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>



=========== QZ Eigen Iteration step 1 =============

-- Left QR similarity update (A = Q*R; A ← R*Q, B ← Qᵀ B Q)

<IPython.core.display.Latex object>



-- Right RQ sweep to restore T (upper triangular)
   Remove bulge from T in row 4 (len=4):


...

   Remove bulge from T in row 2 (len=2):


<IPython.core.display.Latex object>


.... iterate....



<IPython.core.display.Latex object>


A ≈ Q * S * Z' = true
trueQ * T * Z' = 
Q' * Q ≈ I = true
Z' * Z ≈ I = true


----
Observe**:
* The code shows the first step of the QZ iteration, followed by the final result<br>
The optional step 2 reduction to generalized Hessenberg from has been omitted for clarity.
* A similarity transform based on $A$ destroys the zeros in $B$,<br>
consequently each such step is followed by a bulge chasing, applying orthogonal matrices from the left to resore $B$
to upper triangular form.

### 3.3.3 Eigenvalue computation

> The eigenvalue estimation for matrix $S$ is mostly identical to the estimation for the QR eigenvalue problem:
> * detecting $2\times 2$ blocks in $A$ by checking if the off-diagonal element is sufficiently large
> * For each such block, calculating the eigenvalues using the block in $A$ and the corresponding block in $B$,<br>
which might be real or complex depending on the discriminant of the corresponding quadratic equation.<br>
The divisor $\lambda_t$ in this case is the product of the corresponding entries on the diagonal of $T$.
> * If there’s no $2\times 2$ block, it’s a single real eigenvalue.

For the $T$ matrix, the corresponding eigenvalues are the diagonal entries.

In [17]:
%%julia
function estimate_generalized_eigenvalue(A::Matrix{T}, B::Matrix{T}) where T
    """
    Estimate generalized eigenvalues and identify block sizes
    for a quasi-upper-triangular matrix pair (A, B)
    produced by a QZ (generalized Schur) decomposition.

    Returns both the eigenvalues and a vector of block sizes,
    where each entry in `blocks` indicates the dimension of
    its corresponding diagonal block (1 for real, 2 for complex pair).

    Args:
        A::Matrix{T}, B::Matrix{T} : quasi-upper-triangular matrices (same size)

    Returns:
        eigenvalues::Vector{Complex{T}}
        blocks::Vector{Int}  (e.g., [1, 2, 1])
    """
    n = size(A, 1)
    @assert n == size(B, 1) "A and B must have same dimensions"

    λ = Complex{T}[]
    blocks = Int[]
    tol = 1e-12
    i = 1

    while i <= n
        # --- 2×2 block?
        if i < n && abs(A[i+1, i]) > tol
            a11, a12, a21, a22 = A[i,i], A[i,i+1], A[i+1,i], A[i+1,i+1]
            b11, b12, b21, b22 = B[i,i], B[i,i+1], B[i+1,i], B[i+1,i+1]

            # Coefficients of det(A - λB) = 0
            α = b11*b22 - b12*b21
            β = -(a11*b22 + a22*b11 - a12*b21 - a21*b12)
            γ = a11*a22 - a12*a21
            Δ = β^2 - 4*α*γ

            if abs(Δ) < tol
                λ1 = -β / (2*α)
                λ2 = λ1
                append!(blocks, [1,1])
            elseif Δ >= 0
                λ1 = (-β + sqrt(Δ)) / (2*α)
                λ2 = (-β - sqrt(Δ)) / (2*α)
                append!(blocks, [1,1])
            else
                realpart = -β / (2*α)
                imagpart = sqrt(-Δ) / (2*α)
                λ1 = realpart + im*imagpart
                λ2 = realpart - im*imagpart
                push!(blocks, 2)
            end

            append!(λ, [λ1, λ2])
            i += 2
        else
            # --- 1×1 real block
            a_ii, b_ii = A[i,i], B[i,i]
            if abs(b_ii) > tol
                push!(λ, a_ii / b_ii)
            else
                push!(λ, complex(Inf))
            end
            push!(blocks, 1)
            i += 1
        end
    end

    return λ, blocks
end

;

In [19]:
%%julia
function test_qz_eigenproblem()
    #Random.seed!(1234)

    # Example matrices --- B upper-triangular, A random/Hessenberg-like
    n = 4
    A = [1.0 2 3 4;
         5 6 7 8;
         0.1 9 10 11;
         1   0.1 12 13]

    B = [4.0 0.5 0.1 0;
         0.0 3.0 2.1 0.3;
         2.0 0.0 2.0 0.2;
         1.0 -1.0 0.0 1.0]

    py_show("Initial A = ",A, L"\quad B = ",B)
    λ, X = eigen(A, B)

    S = copy(A)
    T = copy(B)
    S, T, Qc, Zc = qz_eigen_iteration!(S, T, 99; debug=false) # avoid the printout out step 100

    λₛ, blocks = estimate_generalized_eigenvalue(S, T)

    py_show("S = ",
        (a=S,
         per_element_style=(x,i,j,fx) ->
             diagonal_blocks_formatter(x,i,j,fx; blocks=blocks)),
        L"\quad T = ",
        (a=T,
         per_element_style=(x,i,j,fx) ->
             diagonal_blocks_formatter(x,i,j,fx; blocks=blocks)),

        number_formatter = x -> round(x, digits = 2)
    )

    py_show( "Naive QZ estimates:",L"\qquad", round.(λₛ,digits=2)' )
    py_show( "Generalized eigenvalues: ", reverse!(round.(λ,digits=2)') )
end
test_qz_eigenproblem()

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

____
**Observe:**
* The QZ algorithm reduces the original matrix pair $(A,B)$ to generalized Schur form $(S,T)$.
* The eigenvalue estimation step reveals two blocks of size $1\times 1$ (real eigenvalues) and a $2\times 2$ block
* The eigenvalue computations agree with the values obtained using a library routine (implementing a far more sophisticated version of the QZ algorithm)

<strong style="color:red;"> FIX: python and julia calls for generalized Schur and QZ</strong>
<div style="display: flex; margin-top: 1em; margin-bottom: 1em;">
<!-- ----------------------------------------- Python box -->
<div style="width: 22em; height: 4.8em; padding-right: 1cm;
            background: #f8f8f8; border: 1px solid #ccc; border-radius: 4px;
            font-family: monospace; font-size: 90%; line-height: 1.2;
            display: flex; align-items: flex-start; padding: 0.6em 0.75em;
            box-sizing: border-box;">
  <div>
    <div style="font-weight: bold; margin-bottom: 0.2em;">Python</div>
    <div>U, _ = np.linalg.qr(A)</div>
    <div>V, _ = np.linalg.qr(B)</div>
  </div>
</div>
<!-- ---------------------------------------- Julia box -->
<div style="width: 22em; height: 4.8em; margin-left: 1cm;
            background: #f8f8f8; border: 1px solid #ccc; border-radius: 4px;
            font-family: monospace; font-size: 90%; line-height: 1.2;
            display: flex; align-items: flex-start; padding: 0.6em 0.75em;
            box-sizing: border-box;">
  <div>
    <div style="font-weight: bold; margin-bottom: 0.2em;">Julia</div>
    <div>U = qr(A).Q</div>
    <div>V = qr(B).Q</div>
  </div>
</div>
</div>

# $\infty$ <strong style="color:red;">ORIGINAL TEXT FROM HERE ON DOWN</strong>

### 3.2.1 Example Pencil

We will introduce the algorithm using the following two matrices:

In [18]:
%%julia
A = [3.0 2.0 1.0; 2.0 3.0 4.0; 1.0 5.0 6.0]
B = [4.0 1.0 2.0; 2.0 3.0 1.0; 1.0 0.0 2.0]
py_show("Let A = ", A, L",\quad B = ", B, number_formatter=x->round_value(x,0))

<IPython.core.display.Latex object>

### 3.2.3 The QZ Algorithm

The first stage in the QZ algorithm applies orthogonal transformations to reduce the matrix pair $(A, B)$  
to a structured intermediate form:
- $A$ is reduced to **upper Hessenberg form** (zero entries below the first subdiagonal),
- $B$ is reduced to **upper triangular form**.

That is, orthogonal matrices $Q_1$ and $Z_1$ are constructed such that:  
$\qquad \displaystyle{ A_1 = Q_1^T A Z_1 }$ is upper Hessenberg  
$\qquad \displaystyle{ B_1 = Q_1^T B Z_1 }$ is upper triangular

* $Q_1$ acts from the left to annihilate entries of $A$ using a QR algorithm  
typically performed using a sequence of [**Householder reflections**](HouseholderReflections.ipynb) or [**Givens Rotations**](GivensRotations.ipynb)  
* $Z_1$ acts from the right to eliminate subdiagonal entries in $B$ using an RQ algorithm as described above.

The same transformations $Q_1$ and $Z_1$ are applied to both $A$ and $B$ to preserve the structure of the generalized eigenproblem.

No assumption is made about symmetry or definiteness of $A$ or $B$.  
The process applies to any square matrices of the same size.

In [ ]:
%%julia
matrices = naive_householder(A)
A₁       = matrices[end][2][:,1:size(A,2)]
Q₁       = matrices[end][2][:,size(A,2)+1:end]'
B₁       = Q₁ * B
h,_=nM.ge( matrices, formater=a -> string(round(a,digits=3)), Nrhs=size(A,1),
    bg_for_entries=[
        [1, 0, [[(0, 0), (2, 2)]], "yellow!30"],[1, 1, [[(0, 0), (2, 0)]], "yellow!50"],
        [2, 0, [[(1, 1), (2, 2)]], "yellow!30"],[2, 1, [[(1, 1), (2, 1)]], "yellow!50"],
    ],
    tmp_dir="/tmp")
h

In [ ]:
%%julia
py_show(L"Q_1 = ", Q₁, L"\quad A_1 = Q_1 A = ", A₁, L"\quad B_1 = Q_1 B = ", B₁, number_formatter=x->round_value(x,3))

In [ ]:
%%julia
A = [3.0 2.0 1.0 4.0;
     2.0 3.0 4.0 1.0;
     1.0 5.0 6.0 -1.0;
     1.0 2.0 1.0 3.0
]

B = [4.0 1.0 2.0 4.0;
     2.0 3.0 1.0 1.0;
     1.0 5.0 2.0 2.0;
    -1.0 3.0 1.0 4.0]
#Z,BZ = right_householder_from_row(B, 1)
Z,BZ=right_reflector_for_column(B,1)
py_show(Z, BZ, B*Z, number_formatter=x->round_value(x,2))

#### Step 2: Hessenberg–Triangular Reduction

The first stage reduces $A$ to upper Hessenberg form and $B$ to upper triangular form:  
$\qquad \displaystyle{ Q_1^T A Z_1 = H }, \qquad \displaystyle{ Q_1^T B Z_1 = R }
$

This can be done via a sequence of **Householder transformations**, similar to the QR algorithm.

#### Step 3: Iterative QZ Step

Next, an iterative process analogous to the QR algorithm is applied:
- A shift $\sigma$ is selected
- The matrix $A - \sigma B$ is implicitly factorized as $QR$
- Update:
  $\; \displaystyle{ A \leftarrow R Q + \sigma, \quad B \leftarrow Q^T B Q }$

Each iteration reduces the subdiagonal structure while maintaining the triangular form of $B$.

The process is repeated until both $A$ and $B$ are simultaneously triangular (or quasi-triangular in the complex case).

---

### Step 4: Eigenvalue Extraction

Once in triangular form  
$\qquad \displaystyle{ S = Q^T A Z }, \quad \displaystyle{ T = Q^T B Z }
$
the eigenvalues of the pencil $A - \lambda B$ are given by  
$\qquad \displaystyle{ \lambda_i = \frac{s_{ii}}{t_{ii}} }, \quad \text{for } t_{ii} \neq 0
$

Infinite or indeterminate eigenvalues may arise if $t_{ii} = 0$.

---

### Summary

- The QZ algorithm generalizes the Schur decomposition by reducing a matrix pair $(A, B)$ to triangular form via orthogonal transformations.
- The process maintains numerical stability and avoids explicit inversion.
- Eigenvalues are obtained from the resulting triangular pencil $(S, T)$.
- Left and right eigenvectors may be computed from the accumulated transformations.

This algorithm is implemented in LAPACK as `xGGEV` and accessed in SciPy via `scipy.linalg.eig`.

For full details, see:
- Golub & Van Loan, *Matrix Computations*, 4th ed., Chapter 7

Let $(A, B)$ be a pair of square matrices of the same size.  
The **generalized Schur decomposition**, also known as the **QZ decomposition**, expresses both matrices in an upper-triangular form via orthogonal (or unitary) similarity transformations.

There exist orthogonal matrices $Q$ and $Z$ such that
$\qquad \displaystyle{ Q^T A Z = S }, \quad \displaystyle{ Q^T B Z = T }$  
where $S$ and $T$ are both upper (quasi-)triangular matrices.

This defines the **generalized Schur form** of the pencil $(A, B)$.

The original generalized eigenproblem  
$\qquad \displaystyle{ A x = \lambda B x }$  
is transformed into the equivalent problem  
$\qquad \displaystyle{ S \hat{x} = \lambda T \hat{x} }$  
with $\hat{x} = Z^{-1} x$.

The generalized eigenvalues are given (formally) as:
$\qquad \displaystyle{ \lambda_i = \frac{s_{ii}}{t_{ii}} }$  
for each $i$ such that $t_{ii} \neq 0$.

If $t_{ii} = 0$ and $s_{ii} \neq 0$, then $\lambda_i = \infty$.  
If both $s_{ii}$ and $t_{ii}$ are zero, the eigenvalue is considered **undefined** or **indeterminate**, and may signal a singular pencil.

---

### Properties

- The eigenvalues of the pencil $(A, B)$ are invariant under the transformation.
- If $A$ and $B$ are both real, then $Q$, $Z$, $S$, and $T$ can be taken as real.
- If the matrices are complex, then $Q$ and $Z$ are unitary.
- The decomposition is **backward stable** and suitable for arbitrary (nonsymmetric, indefinite, or singular) pencils.

---

### Comparison with the Standard Schur Decomposition

- The **standard Schur decomposition** reduces a single matrix to triangular form:  
  $\qquad \displaystyle{ A = Q S Q^T }$
- The **generalized Schur decomposition** reduces a matrix *pair* to simultaneous triangular form via two-sided transformations:
  $\qquad \displaystyle{ (A, B) \mapsto (S, T) = (Q^T A Z, Q^T B Z) }$

This generalization is necessary because no single transformation can, in general, triangularize both $A$ and $B$ simultaneously unless they commute, which is rare in practice.

For arbitrary regular $(A,B)$, compute unitary $Q,Z$ such that  
$\qquad Q^T A Z = S, \quad Q^T B Z = T,$  
with $S,T$ (quasi)upper-triangular. This is the **generalized Schur form**.

- **Eigenvalues** are read from diagonal blocks as ratios $\qquad \lambda_i = S_{ii}/T_{ii}$ (with $T_{ii}=0$ encoding $\lambda_i=\infty$).  
- The orthogonal/unitary steps make QZ **backward stable**: computed results are exact for a nearby $(A+\Delta A,B+\Delta B)$ with small perturbations.

In [ ]:
def qz_eigs(A,B,output='real'):
    if not HAVE_SCIPY:
        raise RuntimeError("SciPy required for QZ.")
    S, T, Q, Z = qz(A, B, output=output)
    alpha = np.diag(S).astype(complex)
    beta  = np.diag(T).astype(complex)
    w = alpha / beta
    return w, (S,T,Q,Z)

# Small demo with a nonsymmetric pair
A = np.array([[4., 2., 0.],
              [1., 3., 1.],
              [0., 0., 2.]])
B = np.array([[1., 0., 0.],
              [0., 2., 0.],
              [0., 0., 1.]])


    def residual(A,B,lam,x):
        num = np.linalg.norm(A@x - lam*(B@x))
        den = (np.linalg.norm(A)*np.linalg.norm(x) + 1e-15)
        return num/den

if HAVE_SCIPY:
    w, (S,T,Q,Z) = qz_eigs(A,B,output='real')
    print("QZ eigenvalues (ratios S_ii/T_ii):", np.real(w))
    # Residual check with right Schur vectors (columns of Z span invariant subspaces)
    for i in range(len(w)):
        x = Z[:,i]
        print(f"i={i}, residual:", np.round(residual(A,B,w[i],x), 3))
else:
    print("SciPy not available; skipping QZ demo.")

# 4. Detecting $\lambda=\infty$ in practice

In QZ form, $T_{ii}\approx 0$ indicates $\lambda_i=\infty$ (since $\lambda_i=S_{ii}/T_{ii}$).

Equivalently, flip to the **reciprocal pencil**
$\;\ B - \mu A \; $
and look for eigenvalues at $\mu=0$.


In [ ]:
# Example: regular pencil with B singular -> one eigenvalue at infinity
A = np.array([[1., 0.],
              [0., 2.]])
B = np.array([[1., 0.],
              [0., 0.]])

if HAVE_SCIPY:
    w1, (S1,T1,_,Z1) = qz_eigs(A,B,output='real')
    print("Eigenvalues of (A,B):", w1, "  (one should be ∞)")
    print("T diag:", np.diag(T1))  # one ~0
    # Reciprocal
    w2, _ = qz_eigs(B,A,output='real')
    print("Eigenvalues of reciprocal (B,A):", w2, "  (expect a zero)")
else:
    print("SciPy not available; ∞-eigenvalue demo skipped.")

# 5. Reordering and deflating subspaces (via `ordqz`)

To extract a **deflating subspace** for a spectral region (e.g., $|\lambda|<1$ for discrete-time stability), reorder the generalized Schur form so that the desired eigenvalues appear first. The corresponding first columns of $Z$ span the deflating subspace.

In [ ]:
if HAVE_SCIPY:
    # Discrete-time example: select |lambda| < 1
    A = np.array([[0.8, 0.3, 0.0],
                  [0.0, 1.2, 0.2],
                  [0.0, 0.0, 0.9]])
    B = np.eye(3)

    def select_inside(alpha, beta):
        w = alpha/beta
        return np.abs(w) < 1.0

    S, T, Q, Z, alpha, beta, sdim = ordqz(A, B, sort=select_inside, output='real')
    w = alpha/beta
    print("Reordered eigenvalues:", w)
    print("Count inside unit disk:", sdim)
    # Extract deflating subspace U = Z[:, :sdim] and verify approximate invariance
    U = Z[:, :sdim]
    PA = U @ np.linalg.lstsq(U, (A@U), rcond=None)[0]
    PB = U @ np.linalg.lstsq(U, (B@U), rcond=None)[0]
    print("||A U - Proj_U(A U)||:", np.linalg.norm(A@U - PA))
    print("||B U - Proj_U(B U)||:", np.linalg.norm(B@U - PB))
else:
    print("SciPy not available; ordqz demo skipped.")


# 6. Backward error viewpoint and residual auditing

A computed pair $(\hat\lambda, \hat x)$ satisfies $A\hat x - \hat\lambda B \hat x = r$. With QZ (orthogonal/unitary updates), the algorithm is **backward stable**: it computes the exact Schur form of a nearby pencil $(A+\Delta A, B+\Delta B)$ with small perturbations.

**Practical check.** Report **normalized residuals**  
$\qquad \dfrac{\|A\hat x - \hat\lambda B\hat x\|}{\|A\|\,\|\hat x\|}$  
for all computed modes; small residuals give confidence in the solution.

In [ ]:
# Residual audit on a random example
np.random.seed(7)
A = np.random.randn(6,6)
B = np.random.randn(6,6)

if HAVE_SCIPY:
    w, (S,T,Q,Z) = qz_eigs(A,B,output='complex')
    res = np.array([residual(A,B,w[i],Z[:,i]) for i in range(len(w))])
    print("Median normalized residual:", np.median(res))
    print("Max normalized residual   :", np.max(res))
else:
    print("SciPy not available; residual audit skipped.")

# 7. Practical recipes and pitfalls

- **If $A=A^T$, $B=B^T \succ 0$**: use the **Cholesky route** (symmetric-definite). You get real eigenvalues and $B$-orthonormal eigenvectors.  
- **Otherwise (regular pencils)**: use **QZ**; read eigenvalues from $S_{ii}/T_{ii}$, detect $\lambda=\infty$ via $T_{ii}\approx 0$.  
- **Never form $B^{-1}A$ explicitly**; prefer triangular solves or Schur/QZ.  
- **Scale/equilibrate** poorly scaled inputs before solving (see Notebook 3).  
- **Always check residuals**; when in doubt, verify with multiple methods (e.g., Cholesky vs QZ) in cases where both apply.

# Take Away

In [ ]:
P      = np.fliplr(np.eye(Main.A.shape[0]))   # permutation that flips order
Qt, Rt = np.linalg.qr(P@ Main.A.T @ P)        # Qt orthogonal, Rt upper‑triangular
B = P @ Rt.T @ P                              # upper‑triangular
C = P @ Qt.T @ P                              # orthogonal

In [ ]:
Main.A - B @ C

# Why the QZ (Generalized Schur) Algorithm Works

## 1.  The Generalized Eigenvalue Problem
Given two square matrices $A,B\in\mathbb{R}^{n\times n}$ (or $\mathbb{C}^{n\times n}$),
the **pencil eigenproblem** seeks scalars $\lambda$ and non‑zero vectors
$x$ such that
$$
A\,x = \lambda\,B\,x .
\tag{1}
$$
Equivalently, $\lambda$ is a root of $\det(A-\lambda B)=0$.

## 2.  Orthogonal (Unitary) Similarities Preserve the Spectrum
For any orthogonal (real) or unitary (complex) matrices $Q,Z$ we have
$Q^{T}Q=I$ and $Z^{T}Z=I$.  Multiplying (1) on the left by $Q^{T}$
and on the right by $Z$ gives
$$
Q^{T}A Z\,\tilde{x}= \lambda\, Q^{T}B Z\,\tilde{x},
\qquad\text{where }\tilde{x}=Z^{T}x .
\tag{2}
$$
Since the map $x\mapsto\tilde{x}=Z^{T}x$ is bijective, $(\lambda,\tilde{x})$ is an
eigenpair of the transformed pencil
$$
\tilde A = Q^{T} A Z,\qquad \tilde B = Q^{T} B Z .
$$
Conversely, any eigenpair of $(\tilde A,\tilde B)$ yields an eigenpair of
$(A,B)$ by the inverse substitution $x = Z\tilde{x}$.  Hence
$$
\sigma_{\text{gen}}(A,B)=\sigma_{\text{gen}}(Q^{T} A Z,\; Q^{T} B Z).
\tag{3}
$$

## 3.  What the Transformations Do

* **Left multiplication** by $Q^{T}$ rotates the **row space**
of both matrices in exactly the same way.
* **Right multiplication** by $Z$ rotates the **column space**
of both matrices in exactly the same way.
* Because the same rotations are applied to **both** members of the
pencil, the **ratio** $A x / B x$ that defines the generalized eigenvalue
remains unchanged.


## 4.  Why Orthogonal/Unitary Matrices
Orthogonal (unitary) matrices satisfy $Q^{T}=Q^{-1}$ and $Z^{T}=Z^{-1}$;
therefore the transformation in (2) is a genuine **similarity**
transformation of the pencil.  Any invertible $S,T$ would preserve the
generalized eigenvalues, but orthogonal/unitary choices have two crucial
advantages:

1. They are norm‑preserving: $\|Q^{T}A Z\|_{2}=\|A\|_{2}$, which keeps
round‑off errors under control.
2. Their condition number is 1, guaranteeing backward stability of the
algorithm.

## 5.  The QZ Iteration
Starting from a Hessenberg–triangular pencil $(H,T)$ (obtained by an
initial reduction), a single QZ step proceeds as follows:

1. Choose a shift $\sigma$ (typically an eigenvalue of the trailing
$2\times2$ block of $(H,T)$).
2. Form the shifted pencil $(H-\sigma T,\;T)$.
3. Apply a Givens rotation $G_{1}$ from the left to zero the sub‑diagonal
element in the first column of $H-\sigma T$; simultaneously multiply both
$H$ and $T$ on the right by $G_{1}^{T}$.  This creates a **bulge**
below the Hessenberg band.
* Chase the bulge down the matrix with successive Givens rotations,
each applied on the left and on the right, until it exits at the bottom‑right
corner.

Every left/right pair $(G_{k}^{T},\,G_{k})$ is orthogonal, so by (3) the
generalized eigenvalues are unchanged.  After a full sweep the trailing
$2\times2$ block becomes (quasi‑)triangular; repeating the sweep drives the
entire pencil to the generalized Schur form
$$
Q^{T} A Z = S,\qquad Q^{T} B Z = T,
$$
with $S$ quasi‑upper‑triangular and $T$ upper‑triangular.  The eigenvalues
are then simply the ratios of corresponding diagonal (or $2\times2$ block)
entries:
$$
\lambda_{i}= \frac{s_{ii}}{t_{ii}}\quad\text{(or the ratio of the
eigenvalues of matching $2\times2$ blocks).}
$$

## 6.  Summary

* Simultaneous left and right orthogonal transformations constitute a
similarity transformation of the matrix pencil, leaving its generalized
eigenvalues invariant.
* The QZ algorithm exploits this property to reshape the pencil
(Hessenberg–triangular $\to$ quasi‑Schur) while preserving the spectrum.
* Orthogonal (unitary) matrices guarantee numerical stability because
they do not amplify norms or condition numbers.